In [1]:
import yfinance as yf
import pandas as pd
import numpy as np # <-- FIX 1: Corrected alias for NumPy
from sqlalchemy import create_engine
from urllib.parse import quote_plus

# --- 1. Get and Prepare the Data ---
tickers = ["AAPL", "NVDA", "TSLA", "GOOGL", "MSFT"]
period = "5y"
print("Downloading stock data...")
data_ohlc = yf.download(tickers, period=period)
print("✅ Data downloaded.")

# --- 2. Calculations (No Changes Needed Here) ---
close_prices = data_ohlc['Close']
sma_df = pd.concat([close_prices.rolling(window=20).mean().add_suffix('_sma20'), 
                    close_prices.rolling(window=50).mean().add_suffix('_sma50')], axis=1)
daily_returns = close_prices.pct_change()
weights = np.array([0.20, 0.20, 0.20, 0.20, 0.20]) # <-- FIX 1: Using np.array
portfolio_df = pd.DataFrame((1 + daily_returns.dot(weights)).cumprod(), columns=['cumulative_return'])
total_returns = (1 + daily_returns).cumprod().iloc[-1]
gainers_losers_df = pd.DataFrame(total_returns.sort_values(ascending=False), columns=['total_return_factor'])
print("✅ All calculations are complete.")

# --- 3. Prepare the Original OHLC Data for SQL ---
# NEW: Reshape the data from wide to long format for database-friendliness
ohlc_long_format = data_ohlc.stack().reset_index().rename(columns={'level_1': 'Ticker'})
print("✅ OHLC data reshaped for loading.")

# --- 4. Connect to PostgreSQL Database ---
db_user = 'postgres'
db_password = 'post123'
db_host = 'localhost'
db_port = '5432'
db_name = 'postgres'
connection_string = f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"
engine = create_engine(connection_string)
print("Database engine created.")

# --- 5. Load All DataFrames into the 'Stockprice' Schema ---
schema_name = 'Stockprice' 
try:
    print(f"\nLoading data into schema: '{schema_name}'...")
    
    # NEW: Load the ORIGINAL OHLC data
    ohlc_long_format.to_sql('daily_ohlc_prices', engine, schema=schema_name, if_exists='replace', index=False)
    print(f"✅ Table 'daily_ohlc_prices' loaded.")
    
    # Load SMAs, Portfolio, and Ranking tables (as before)
    sma_df.to_sql('simple_moving_averages', engine, schema=schema_name, if_exists='replace', index=True)
    print(f"✅ Table 'simple_moving_averages' loaded.")
    portfolio_df.to_sql('portfolio_performance', engine, schema=schema_name, if_exists='replace', index=True)
    print(f"✅ Table 'portfolio_performance' loaded.")
    gainers_losers_df.to_sql('stock_performance_ranking', engine, schema=schema_name, if_exists='replace', index=True)
    print(f"✅ Table 'stock_performance_ranking' loaded.")
    
    print("\nAll data has been successfully loaded.")
except Exception as e:
    print(f"🔥 An error occurred: {e}")

C:\Users\Jossie-pc\AppData\Local\Temp\ipykernel_7336\159188903.py:11: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data_ohlc = yf.download(tickers, period=period)
[*********************100%***********************]  5 of 5 completed


✅ Data downloaded.
✅ All calculations are complete.
✅ OHLC data reshaped for loading.


C:\Users\Jossie-pc\AppData\Local\Temp\ipykernel_7336\159188903.py:27: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  ohlc_long_format = data_ohlc.stack().reset_index().rename(columns={'level_1': 'Ticker'})


Database engine created.

Loading data into schema: 'Stockprice'...
✅ Table 'daily_ohlc_prices' loaded.
✅ Table 'simple_moving_averages' loaded.
✅ Table 'portfolio_performance' loaded.
✅ Table 'stock_performance_ranking' loaded.

All data has been successfully loaded.
